In [1]:
import os, sys, re, json, time, unittest
import itertools, collections
import re
import operator
import numpy as np
import nltk
import math
import pandas as pd
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
stop_words = set(stopwords.words("english"))
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

/anaconda2/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
#import csv as pandas array
#text_train_deps = pd.read_csv("text_train_deps.csv", encoding='utf8')
text_train_deps = pd.read_csv("text_train_deps.csv")


text_train_deps.head(5)

train_sentences = []
#combines all columns into 1 string
n = len(text_train_deps)
for row in text_train_deps['description'][:n].map(str) + ';'+ text_train_deps['host_about'][:n].map(str) + ';'+ text_train_deps['house_rules'][:n].map(str) + ';'+ text_train_deps['neighborhood_overview'][:n].map(str) + ';'+ text_train_deps['notes'][:n].map(str) + ';'+ text_train_deps['summary'][:n].map(str):
    train_sentences.append(row)
print len(train_sentences)
print len(text_train_deps)

127322
127322


In [3]:
for each in train_sentences[:5]:
    print each

我的房源靠近San Francisco Station, AT&T Park,Safeway,。我的房源适合情侣、独自旅行的冒险家、商务旅行者、有小孩的家庭、毛茸茸的伙伴(宠物)。;nan;nan;nan;nan;我的房源靠近San Francisco Station, AT&T Park,Safeway,。我的房源适合情侣、独自旅行的冒险家、商务旅行者、有小孩的家庭、毛茸茸的伙伴(宠物)。
a "Private Bedroom" in a classic Victorian one block from famous 24th Street in Noe Valley, easy to access the J Church Municipal Railway and the BART system. Best Location you will find in the City! We are steps from 24th Street in the heart of Noe Valley. When family is not staying, we would like to offer up our home to guests traveling through San Francisco - the most beautiful city in the world.  We suggest two people because we are planning to use/rent out other spaces within our home; however, if you have a more than than four, we can make arrangements to rent the entire top floor, which is equivalent to a flat. Access to the dining room, kitchen, and bathroom.  The downstairs is a live/work space that is occupied by the owner. I'd love to hear about your journeys and may be available 

In [4]:
#creating entity features

corpus = train_sentences

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

#getting tf and word count
document_entity_features = []

for row in corpus:
    sentences = nltk.sent_tokenize(row.decode('ascii', 'ignore'))
    token_sent = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged = [nltk.pos_tag(sentence) for sentence in token_sent]
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    
    for tree in chunked:
        document_entity_features.append(extract_entity_names(tree))

In [5]:
#accumulate the "feature appearance" in document counts and feature word counts
document_count = {}
common_feature_words = {}
for doc in document_entity_features:
    current_document_count = {}
    for word in doc:
        if word.lower() not in stop_words:
            #gets the most common feature words
            if word.lower() not in common_feature_words:
                common_feature_words[word.lower()] = 1
            else:
                common_feature_words[word.lower()] += 1
            
    #accumulates the appearance of word in documents
            if word.lower() not in current_document_count:
                current_document_count[word.lower()] = 1

    for key in current_document_count.keys():
        if key not in document_count:
            document_count[key] = current_document_count[key]
        elif key in document_count:
            document_count[key] += current_document_count[key]
            

In [99]:
#pulls the least common 10000 words
#set the number of words
n = 10000

print len(common_feature_words)
keys = sorted(common_feature_words.items(), key=operator.itemgetter(1))
sorted_keys = [str(key[0]) for key in keys]
least_common_keys = sorted_keys[:n]
#print common_feature_words
least_common_entities = {}
for key in least_common_keys:
    least_common_entities[key] = common_feature_words[key]

print len(least_common_entities)
print least_common_entities


25989
10000
{'three bedroom one bath urban oasis': 8, 'blue plate emmy': 2, 'fat buddha': 2, 'experience san': 1, 'pizza clay oven': 1, 'othe mill': 3, 'francesco': 2, 'summary charming': 2, 'gabrielle': 1, 'no pets behave goodly': 5, 'wisconsin jen': 6, 'kitchennett': 3, 'city registration number': 1, 'san francisco civic arts center': 2, 'west africa': 1, 'laundry please': 5, 'alamo square/divisadero': 8, 'kearney street': 1, 'sony hdtv': 2, 'guests alex': 1, 'small refrigerator': 1, 'tolga': 1, 'dell': 6, 'cityfisherman': 8, 'ord ct': 2, 'shanghai explorer': 1, 'van ness street': 5, 'philips hue light': 4, 'sap labs': 1, 'royale rumble go': 7, 'asking about parking': 2, 'market moscone': 1, 'guardsmen': 3, 'page bar': 8, 'eagle': 2, 'vanina': 2, 'ornate': 8, 'christine inexpensive': 8, 'key to': 1, 'attended': 2, 'french lycee': 1, 'sxsw festival': 5, 'hip potrero hill location': 2, 'brexit': 3, 'dakshin': 2, 'sparks food truck': 2, 'bittorrent': 1, 'visitation vall': 4, 'tower fan'

In [100]:
sid = SentimentIntensityAnalyzer()
def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

#getting tf and word count
total_word_count = 0
tfidf_sentiment = []
#for pandas export purposes
tfidf = []
sentiment = []

for row in corpus:
    current_entities = []
    temp_tfidf = []
    sentences = nltk.sent_tokenize(row.decode('ascii', 'ignore'))
    token_sent = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged = [nltk.pos_tag(sentence) for sentence in token_sent]
    chunked = nltk.ne_chunk_sents(tagged, binary=True)
    
    for tree in chunked:
        current_entities.extend(extract_entity_names(tree))
    #gets chunked word count
    for word in sentence:
        total_word_count += 1   
        
    if len(current_entities) > 0:
        
        #counts the amount of times the word appears in the sentence
        current_word_count = {}
        for word in current_entities:
            lower_word = word.lower()
            if lower_word in least_common_entities:
                if lower_word not in current_word_count:
                    current_word_count[lower_word] = 1
                elif lower_word in current_word_count:
                    current_word_count[lower_word] += 1
        if len(current_word_count) > 0:    
            
            for word in current_word_count.keys():
                tf = float(current_word_count[word])/total_word_count
                idf = math.log(len(corpus)/(1+float(document_count[word])))
                tf_idf = tf*idf
                temp_tfidf.append([word,tf_idf])
        else:
                temp_tfidf.append(["no_keys", str(0.0)])
    else:
        temp_tfidf.append(["no_keys", str(0.0)])
        
    ss = sid.polarity_scores(row)
    tfidf_sentiment.append([temp_tfidf, ss])    
    
    #for exporting pandas purposes
    tfidf.append(temp_tfidf)
    sentiment.append(ss)

In [101]:
print len(tfidf_sentiment)
#for each in tfidf_sentiment:
#    print each

127322


In [102]:
for each in tfidf_sentiment[:1000]:
    print each

[[['no_keys', '0.0']], {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
[[['no_keys', '0.0']], {'neg': 0.017, 'neu': 0.823, 'pos': 0.16, 'compound': 0.9972}]
[[['no_keys', '0.0']], {'neg': 0.039, 'neu': 0.903, 'pos': 0.057, 'compound': 0.7461}]
[[['no_keys', '0.0']], {'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'compound': 0.872}]
[[['no_keys', '0.0']], {'neg': 0.062, 'neu': 0.745, 'pos': 0.193, 'compound': 0.9965}]
[[['no_keys', '0.0']], {'neg': 0.009, 'neu': 0.802, 'pos': 0.188, 'compound': 0.9987}]
[[['no_keys', '0.0']], {'neg': 0.0, 'neu': 0.784, 'pos': 0.216, 'compound': 0.9943}]
[[[u'kaushick five', 0.05724871626061517]], {'neg': 0.039, 'neu': 0.846, 'pos': 0.115, 'compound': 0.9929}]
[[[u'mission garden', 0.05217321510993875], [u'smoking please', 0.05299316553455183]], {'neg': 0.04, 'neu': 0.856, 'pos': 0.104, 'compound': 0.9744}]
[[['no_keys', '0.0']], {'neg': 0.041, 'neu': 0.699, 'pos': 0.261, 'compound': 0.9993}]
[[['no_keys', '0.0']], {'neg': 0.022, 'neu': 0.852, 'pos': 0.

[[['no_keys', '0.0']], {'neg': 0.042, 'neu': 0.785, 'pos': 0.173, 'compound': 0.9959}]
[[[u'spoiler alert', 0.0008694986141120391]], {'neg': 0.051, 'neu': 0.755, 'pos': 0.194, 'compound': 0.9974}]
[[[u'sam francisco', 0.0008422593408238848]], {'neg': 0.017, 'neu': 0.653, 'pos': 0.33, 'compound': 0.995}]
[[['no_keys', '0.0']], {'neg': 0.0, 'neu': 0.901, 'pos': 0.099, 'compound': 0.6808}]
[[['no_keys', '0.0']], {'neg': 0.032, 'neu': 0.775, 'pos': 0.194, 'compound': 0.9907}]
[[['no_keys', '0.0']], {'neg': 0.0, 'neu': 0.712, 'pos': 0.288, 'compound': 0.9951}]
[[['no_keys', '0.0']], {'neg': 0.017, 'neu': 0.809, 'pos': 0.173, 'compound': 0.9895}]
[[['no_keys', '0.0']], {'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.9919}]
[[['no_keys', '0.0']], {'neg': 0.045, 'neu': 0.765, 'pos': 0.191, 'compound': 0.978}]
[[['no_keys', '0.0']], {'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'compound': 0.9638}]
[[['no_keys', '0.0']], {'neg': 0.046, 'neu': 0.697, 'pos': 0.257, 'compound': 0.9994}]
[[['no_k

In [124]:
from pandas import DataFrame

pd_tfidf_sentiment = DataFrame.from_records(tfidf_sentiment)
pd_tfidf_sentiment.columns = ["keys", "sentiment"]
pd_tfidf_sentiment.head(5)

pd_tfidf_sentiment.to_csv('pd_tfidf_sentiment.csv', encoding='utf-8')

In [129]:
test = pd.read_csv("pd_tfidf_sentiment.csv")
test.head(5)
print len(text_train_deps)
print len(test)

127322
127322
